In [0]:
!pip install glove-python

In [0]:
#GloVe 실습

from __future__ import print_function
import argparse
import pprint
import gensim

from glove import Glove
from glove import Corpus
import numpy as np

In [0]:
import nltk  
nltk.download('stopwords')

#import nltk.downloader as api
from nltk.corpus import stopwords  
stops = stopwords.words('english')
stops[:10]  

In [0]:
# Define character set to delete

delchars = [chr(c) for c in range(256)] 
delchars = [x for x in delchars if not x.isalnum()]
delchars.remove(' ')
delchars = ''.join(delchars)

print (delchars)

In [0]:
import gensim.downloader as api

corpus = api.load('text8')


In [0]:
!wget https://github.com/hanjy1777/KAIST-NLP-tutorial/raw/master/part2/shakespeare.txt
#!wget https://github.com/hanjy1777/KAIST-NLP-tutorial/raw/master/part2/tv_text.txt

filename = 'shakespeare.txt'
#filename= 'enwiki-latest-pages-articles9.xml-p1791081p2336422.bz2'
# Make the corpus lowercase and delete non-letter or non-digit. 
# Tokenize with space.

corpus = []
with open(filename, 'r') as datafile:
    for line in datafile:
        tmp = [x for x in line.lower().translate(None, delchars).split(' ') if x != '']
        if tmp != []: corpus.append(tmp)


In [0]:
corpus[4]

In [0]:
# Build the corpus dictionary and co-occurrence matrix
corpus_model = Corpus()
corpus_model.fit(corpus, window = 10)
print('Dict size: %s' % len(corpus_model.dictionary))
print('Collocations: %s' % corpus_model.matrix.nnz)

In [0]:
epochs = 30
ndim = 50

glove = Glove(no_components=ndim, learning_rate=0.2) # 100 dimensional embedding
glove.fit(corpus_model.matrix, epochs=epochs,
          no_threads=1, verbose=True)
glove.add_dictionary(corpus_model.dictionary)

#glove.save('glove.model')

In [0]:
for word in ['romeo', 'king', 'face']:
    print(word)
    sim_words = glove.most_similar(word, number=10)
    for w in sim_words:
        print (w)
    
    print("")


In [0]:
# randomly show the words and its closest words
import random

words = random.sample(corpus_model.dictionary.keys(),3)

for word in words:
    print(word)
    sim_words = glove.most_similar(word, number=10)
    for w in sim_words:
        print (w)
    
    print("")


In [0]:
!wget https://github.com/aaronkub/machine-learning-examples/raw/master/imdb-sentiment-analysis/movie_data.tar.gz

In [0]:
reviews_train = []
for line in open('full_train.txt', 'r'):
    reviews_train.append(line.strip())
    
reviews_test = []
for line in open('full_test.txt', 'r'):
    reviews_test.append(line.strip())
    
print(reviews_train[0])

In [0]:
# Preprocess the texts
import re

REPLACE_NO_SPACE = re.compile("[.;:!\'?,\"()\[\]]")
REPLACE_WITH_SPACE = re.compile("(<br\s*/><br\s*/>)|(\-)|(\/)")

def preprocess_reviews(reviews):
    reviews = [REPLACE_NO_SPACE.sub("", line.lower()) for line in reviews]
    reviews = [REPLACE_WITH_SPACE.sub(" ", line) for line in reviews]
    
    return reviews

reviews_train_clean = preprocess_reviews(reviews_train)
reviews_test_clean = preprocess_reviews(reviews_test)

print(reviews_train_clean[0])

In [0]:
# Build dataset

X = []
X_norm = []
for data in reviews_train_clean:
    word_vec = np.array([0.0]*ndim)
    word_cnt = 0
    for word in data:
        if word in glove.dictionary:
            word_vec += glove.word_vectors[glove.dictionary[word],:]
            word_cnt += 1
    
    X.append(word_vec)
    X_norm.append(word_vec/word_cnt)
    

print("The number of train data: %d"%len(X))

In [0]:
X_test = []
for data in reviews_train_clean:
    word_vec = np.array([0.0]*ndim)
    #word_cnt = 0
    for word in data:
        if word in glove.dictionary:
            word_vec += glove.word_vectors[glove.dictionary[word],:]
            #word_cnt += 1
    
    X_test.append(word_vec)
    #X_norm.append(word_vec/word_cnt)
print("The number of test data: %d"%len(X_test))

In [0]:
from keras.models import Model, Input
from keras.layers import LSTM, Embedding, Dense, TimeDistributed, Dropout, Bidirectional
from keras_contrib.layers import CRF

# Model definition
input = Input(shape=(MAX_LEN,))



model = Embedding(input_dim=n_words+2, output_dim=EMBEDDING, # n_words + 2 (PAD & UNK)
                  input_length=MAX_LEN, mask_zero=True)(input)  # default: 20-dim embedding
model = Bidirectional(LSTM(units=50, return_sequences=True,
                           recurrent_dropout=0.1))(model)  # variational biLSTM
model = TimeDistributed(Dense(50, activation="relu"))(model)  # a dense layer as suggested by neuralNer
crf = CRF(n_tags+1)  # CRF layer, n_tags+1(PAD)
out = crf(model)  # output

model = Model(input, out)
model.compile(optimizer="rmsprop", loss=crf.loss_function, metrics=[crf.accuracy])

model.summary()

In [0]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split

target = [1 if i < 12500 else 0 for i in range(25000)]

X_train, X_val, y_train, y_val = train_test_split(
    X, target, train_size = 0.75
)

for c in [0.01, 0.05, 0.25, 0.5, 1]:
    
    lr = LogisticRegression(C=c)
    lr.fit(X_train, y_train)
    print ("Accuracy for C=%s: %s" 
           % (c, accuracy_score(y_val, lr.predict(X_val))))



In [0]:
final_model = LogisticRegression(C=0.5)
final_model.fit(X, target)
print ("Final Accuracy: %s" 
       % accuracy_score(target, final_model.predict(X_test)))

In [0]:
pred = final_model.predict_proba(X_test)

In [0]:
# The most discriminating features

feature_to_coef = {
    word: coef for word, coef in zip(
        range(100), final_model.coef_[0]
    )
}

print ("Best positive features")

for best_positive in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1], 
    reverse=True)[:5]:
    print (best_positive)
print ()
    
#     ('excellent', 0.9288812418118644)
#     ('perfect', 0.7934641227980576)
#     ('great', 0.675040909917553)
#     ('amazing', 0.6160398142631545)
#     ('superb', 0.6063967799425831)
print ("Best negative features")
    
for best_negative in sorted(
    feature_to_coef.items(), 
    key=lambda x: x[1])[:5]:
    print (best_negative)
print ()
    
#     ('worst', -1.367978497228895)
#     ('waste', -1.1684451288279047)
#     ('awful', -1.0277001734353677)
#     ('poorly', -0.8748317895742782)
#     ('boring', -0.8587249740682945)

In [0]:
from matplotlib import pyplot as plt

_,coef = zip(*feature_to_coef.items())
plt.stem(coef)

In [0]:
from keras.models import Sequential
from keras.layers import Dense, Embedding, LSTM, GRU
from keras.layers.embeddings import Embedding
from keras import optimizers

In [0]:
EMBEDDING_DIM = 50

In [0]:
print('Build model...')

In [0]:
model = Sequential()
model.add(Embedding(vocab_size, EMBEDDING_DIM, input_length=max_length))
model.add(GRU(units=32, dropout=0.2, recurrent_dropout=0.2))
model.add(Dense(1, activation='sigmoid'))


sgd = optimizers.SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='binary_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [0]:
model.summary()

In [0]:
VALIDATION_SPLIT=0.2
indices = np.arange(review_pad.shape[0])

In [0]:
print('Train...')

model.fit(X_train_pad, target, batch_size=128, epochs=2, validation_data=(X_test_pad, target), verbose=2)